In [411]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import vectorbt as vbt
import time
import ccxt

pd.set_option('display.width', 1000)

In [421]:
# 获取交易所数据 使用ccxt库，ccxt 支持多个交易所的加密货币交易库 安装 pip install ccxt
print(ccxt.exchanges)
print(len(ccxt.exchanges))

# 以binance为例子
exchange = ccxt.binance({
    'apiKey': '',
    'secret': '',
    'timeout': 30000,
    'options': {'defaultType': 'spot'},
    'proxies': {
        'http': 'http://127.0.0.1:7890',
        'https': 'http://127.0.0.1:7890',
    },
})
# exchange.load_markets()

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitteam', 'bitvavo', 'bl3p', 'blockchaincom', 'blofin', 'btcalpha', 'btcbox', 'btcmarkets', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseadvanced', 'coinbaseexchange', 'coinbaseinternational', 'coincheck', 'coinex', 'coinlist', 'coinmate', 'coinmetro', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hashkey', 'hitbtc', 'hollaex', 'htx', 'huobi', 'huobijp', 'hyperliquid', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okx', 'onetrading', 'oxfun', 'p2b', 

In [413]:
#  获取ohlcv
df = pd.DataFrame(exchange.fetch_ohlcv(symbol='BTC/USDT', timeframe='1m', limit=1000))
print(df)
df.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df

                 0         1         2         3         4         5
0    1729630980000  67523.18  67523.19  67492.00  67492.00   4.66687
1    1729631040000  67492.00  67492.01  67480.00  67480.01   2.17951
2    1729631100000  67480.00  67480.01  67464.00  67464.01   2.61444
3    1729631160000  67464.00  67472.59  67444.60  67444.60   6.48562
4    1729631220000  67444.61  67459.25  67444.50  67459.25   6.43580
..             ...       ...       ...       ...       ...       ...
995  1729690680000  66508.68  66535.24  66494.82  66514.62  13.24835
996  1729690740000  66514.62  66552.11  66512.26  66530.37  22.67383
997  1729690800000  66530.37  66566.85  66526.00  66566.85  20.08157
998  1729690860000  66566.85  66575.76  66550.00  66575.76   9.36808
999  1729690920000  66575.75  66610.00  66575.75  66604.98  19.89889

[1000 rows x 6 columns]


,t,o,h,l,c,v
0,1729630980000,67523.18,67523.19,67492.00,67492.00,4.66687
1,1729631040000,67492.00,67492.01,67480.00,67480.01,2.17951
2,1729631100000,67480.00,67480.01,67464.00,67464.01,2.61444
3,1729631160000,67464.00,67472.59,67444.60,67444.60,6.48562
4,1729631220000,67444.61,67459.25,67444.50,67459.25,6.43580
...,...,...,...,...,...,...
995,1729690680000,66508.68,66535.24,66494.82,66514.62,13.24835
996,1729690740000,66514.62,66552.11,66512.26,66530.37,22.67383
997,1729690800000,66530.37,66566.85,66526.00,66566.85,20.08157
998,1729690860000,66566.85,66575.76,66550.00,66575.76,9.36808


In [414]:
#
class StrategyFrame():
    df = pd.DataFrame()
    orders = pd.DataFrame()
    exchange = ccxt.binance({
        'apiKey': '',
        'secret': '',
        'timeout': 30000,
        'options': {'defaultType': 'spot'},
        'proxies': {
            'http': 'http://127.0.0.1:7890',
            'https': 'http://127.0.0.1:7890',
        },
    })
    #初始化
    def __init__(self, data: pd.DataFrame):
        self.df = data
    # 数据按时间聚合
    def resample_data(self, rule='15min'):
        res_df = pd.DataFrame()
        res_df['t'] = self.df['o'].resample(rule).last()
        res_df['o'] = self.df['o'].resample(rule).first()
        res_df['h'] = self.df['h'].resample(rule).max()
        res_df['l'] = self.df['l'].resample(rule).min()
        res_df['c'] = self.df['c'].resample(rule).last()
        res_df['v'] = self.df['v'].resample(rule).sum()
        self.df = res_df
        return self
    # 获取最近的数据，要求在1000分钟之内
    def get_last_diff_data(self):
        since = self.df['t'].tail(1).values[0]
        print(f'since: {since}',pd.to_datetime(since,unit='ms',origin='1970-01-01 08:00:00'))
        res_df = pd.DataFrame(exchange.fetch_ohlcv(symbol='BTC/USDT', timeframe='1m',since=since))
        res_df.columns = ['t', 'o', 'h', 'l', 'c', 'v']
        res_df['datetime'] = pd.to_datetime(res_df['t'],unit='ms',origin='1970-01-01 08:00:00')
        res_df.set_index('datetime', inplace=True)
        self.df = pd.concat([self.df[:-1],res_df])
        return self
    # 获取最近几K条数据更新数据并存入csv
    def get_last_data(self,count=1):
        df = pd.DataFrame(exchange.fetch_ohlcv(symbol='BTC/USDT', timeframe='1m', limit=1000))
        for i in range(count - 1):
            since = df[0][0]-1000*60*1000
            res_df = pd.DataFrame(exchange.fetch_ohlcv(symbol='BTC/USDT',timeframe='1m',since=since,limit=1000))
            df = pd.concat([res_df, df], ignore_index=True)
            time.sleep(0.2)
        df.columns = ['t', 'o', 'h', 'l', 'c', 'v']
        df['datetime'] = pd.to_datetime(df['t'],unit='ms',origin='1970-01-01 08:00:00')
        df.set_index('datetime', inplace=True)
        #
        if self.check_data(df):
            df.to_csv('./BTCUSDT.csv')
            self.df = df
        else:
            print('check data error,no update')
        return self
    #检查数据完整性 True完整 False不完整
    def check_data(self,data: pd.DataFrame):
        duplicated_ok = data[data.index.duplicated()].empty
        diff = data['t'].diff()[1:]
        diff_ok = len(diff[diff==diff.iloc[0]])==len(diff)
        if not duplicated_ok:
            print('数据有重复')
        if not diff_ok:
            print('数据不连续')
        return duplicated_ok and diff_ok

    # 生成交易信号, 交易信号的返回内容有 index date o h l c v entry exit
    # 假设模拟最简单的低买高卖， 低于等于低价 即买入，高于高价即卖出
    def signal(self,min_price,max_price):
        entry_list = {}
        exit_list = {}
        res_df = pd.DataFrame()
        res_df['t'] = self.df['t']
        res_df['o'] = self.df['o']
        res_df['h'] = self.df['h']
        res_df['l'] = self.df['l']
        res_df['c'] = self.df['c']
        res_df['v'] = self.df['v']
        res_df['date'] = self.df.index
        for i, (index, row) in enumerate(res_df.iterrows()):
            entry_list[index] = False
            exit_list[index] = False
            #入场
            if row['c']<=min_price:
                entry_list[index] = True
            if row['c']>=max_price:
                exit_list[index] = True
        #
        res_df['entry'] = entry_list.values()
        res_df['exit'] = exit_list.values()
        self.df = res_df
        return self

    def signal_ma(self, N1=5, N2=20):
        entry_list = {}
        exit_list = {}
        res_df = pd.DataFrame()
        res_df['t'] = self.df['t']
        res_df['o'] = self.df['o']
        res_df['h'] = self.df['h']
        res_df['l'] = self.df['l']
        res_df['c'] = self.df['c']
        res_df['v'] = self.df['v']

        res_df['date'] = self.df.index
        res_df['ma_s'] = res_df['c'].rolling(N1, min_periods=1).mean()
        res_df['ma_l'] = res_df['c'].rolling(N2, min_periods=1).mean()
        last_ma_s = 0
        last_ma_l = 0
        ma_100 = res_df['c'].rolling(100).mean()
        # 默认等交易数据N2根bar以后再交易
        for i, (index, row) in enumerate(res_df.iterrows()):
            entry_list[index] = False
            exit_list[index] = False
            #入场
            if i == N1:
                last_ma_s = row['ma_s']
            if i == N2:
                last_ma_l = row['ma_l']
            if i > N2:
                if (row['ma_s'] >= row['ma_l']) and (last_ma_s < last_ma_l):
                    limit_price = res_df.loc[res_df.index[i - 1], 'c']
                    if row['c']<limit_price:
                        entry_list[index] = True
                if (row['ma_s'] <= row['ma_l']) and (last_ma_s > last_ma_l):
                    exit_list[index] = True
                last_ma_s = row['ma_s']
                last_ma_l = row['ma_l']
        #
        res_df['entry'] = entry_list.values()
        res_df['exit'] = exit_list.values()
        self.df = res_df
        return self

    # vectorbt快速回测
    def backtest(self,init_cash=100,fees=0.001):
        pf = vbt.Portfolio.from_signals(
            self.df['c'],
            entries=self.df['entry'],
            exits=self.df['exit'],
            price=self.df['c'],
            init_cash=init_cash,
            fees=fees
        )
        return pf
    # 展示k线
    def show_kline(self):
        res_df = self.df.copy()
        fig = go.Figure(layout=go.Layout(width=1200, height=600))
        fig.add_candlestick(
            name='kline',
            x=res_df.index,
            open=res_df['o'],
            high=res_df['h'],
            low=res_df['l'],
            close=res_df['c']
        )
        fig.data[0].increasing.fillcolor = 'green'
        fig.data[0].increasing.line.color = 'green'
        fig.data[0].decreasing.fillcolor = 'red'
        fig.data[0].decreasing.line.color = 'red'
        # #添加MA_S MA_L
        # fig.add_scatter(x=res_df.index, y=res_df['ma_s'], mode='lines', line=dict(color='blue', width=1), name="MA_S")
        # fig.add_scatter(x=res_df.index, y=res_df['ma_l'], mode='lines', line=dict(color='orange', width=1), name="MA_L")
        fig.update(layout_xaxis_rangeslider_visible=False)
        fig.show()


In [418]:
# 回测阶段，参数调优，不需要get_last_data
data = pd.read_csv('./BTCUSDT.csv', parse_dates=['datetime'])
data.set_index('datetime', inplace=True)
param_list = []
for x in range(3, 11):
    for y in range(14, 31):
        SF = StrategyFrame(data).resample_data('30min').signal_ma(x,y)
        pf = SF.backtest(init_cash=100,fees=0.001)
        winrate = pf.stats()[r'Win Rate [%]']
        total_return = pf.stats()[r'Total Return [%]']
        total_profit = pf.total_profit()
        max_drawdown = pf.max_drawdown()
        total_trades = len(pf.trades)
        # print(f'{x} {y} total_return: {total_return} winrate: {winrate} total_trades: {total_trades} max_drawdown: {max_drawdown}')
        param_list.append((x, y, total_return, total_trades, winrate))

np.array(param_list)

array([[  3.        ,  14.        ,  -4.01023336,  41.        ,
         21.95121951],
       [  3.        ,  15.        ,  -0.60934634,  40.        ,
         25.        ],
       [  3.        ,  16.        ,  -2.64591877,  39.        ,
         25.64102564],
       [  3.        ,  17.        ,  -2.18359814,  36.        ,
         25.        ],
       [  3.        ,  18.        ,  -4.23480956,  39.        ,
         20.51282051],
       [  3.        ,  19.        ,  -8.2059486 ,  38.        ,
         21.05263158],
       [  3.        ,  20.        ,  -5.37706616,  39.        ,
         20.51282051],
       [  3.        ,  21.        ,  -3.41301613,  39.        ,
         23.07692308],
       [  3.        ,  22.        ,  -3.17352178,  29.        ,
         20.68965517],
       [  3.        ,  23.        ,  -9.97793339,  31.        ,
         16.12903226],
       [  3.        ,  24.        ,  -7.66490442,  33.        ,
         18.18181818],
       [  3.        ,  25.        , -10.489

In [419]:
SF = StrategyFrame(data).resample_data('1min').signal(60000,66000)
pf = SF.backtest(init_cash=100,fees=0.001)
pf.stats()

Start                         2024-08-15 10:25:00
End                           2024-10-23 21:04:00
Period                           69 days 10:40:00
Start Value                                 100.0
End Value                              124.378121
Total Return [%]                        24.378121
Benchmark Return [%]                    13.829013
Max Gross Exposure [%]                      100.0
Total Fees Paid                          0.450506
Max Drawdown [%]                        18.946801
Max Drawdown Duration            31 days 14:49:00
Total Trades                                    2
Total Closed Trades                             2
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                100.0
Best Trade [%]                           13.06463
Worst Trade [%]                         10.028935
Avg Winning Trade [%]                   11.546783
Avg Losing Trade [%]                          NaN


In [422]:
# 以上简单策略如何实盘
# 循环获取最新的数据，合成想要的数据，进行交易信号生成，生成订单，根据订单进行买卖，保存订单记录到csv，，然后等待到执行下个循环
# SF = StrategyFrame(pd.DataFrame()).get_last_data(100)
data = pd.read_csv('./BTCUSDT.csv', parse_dates=['datetime'])
data.set_index('datetime', inplace=True)
SF = StrategyFrame(data)
while True:
    SF = SF.get_last_diff_data().signal(60000,66000)
    print(SF.df.tail())
    pf = SF.backtest(init_cash=100,fees=0.001)
    orders = pf.orders.records_readable
    print(orders.tail())
    side = orders.tail(1).Side.values[0]
    price = orders.tail(1).Price.values[0]
    print(side,price)
    #执行交易逻辑
    time.sleep(61)




since: 1729688640000 2024-10-23 21:04:00
                                 t         o         h         l         c         v                date  entry  exit
datetime                                                                                                             
2024-10-23 22:31:00  1729693860000  66727.28  66727.28  66700.94  66718.00   6.57605 2024-10-23 22:31:00  False  True
2024-10-23 22:32:00  1729693920000  66717.99  66755.99  66711.17  66754.02  14.67072 2024-10-23 22:32:00  False  True
2024-10-23 22:33:00  1729693980000  66754.01  66790.00  66736.02  66789.99  12.79741 2024-10-23 22:33:00  False  True
2024-10-23 22:34:00  1729694040000  66789.99  66824.00  66789.99  66822.00  15.88418 2024-10-23 22:34:00  False  True
2024-10-23 22:35:00  1729694100000  66822.00  66876.39  66822.00  66876.38  10.14582 2024-10-23 22:35:00  False  True
   Order Id  Column           Timestamp      Size     Price      Fees  Side
0         0       0 2024-08-15 10:25:00  0.001713  58313.

KeyboardInterrupt: 